<a href="https://colab.research.google.com/github/NathanR100/Example-Code-Projects/blob/main/AOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#nathan's API
WLSACCESSID = 'b678733f-8704-4879-b97d-d74abc85d650'
WLSSECRET = '432e4445-821c-4877-89ee-67f7d68bf4d4'
LICENSEID = 2433393

In [ ]:
!pip install gurobipy

import gurobipy as gp
import re
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 46.9 MB/s eta 0:00:00


In [ ]:
class AOP:
  def __init__(self):
    nr_segments = 0
    nr_products = 0
    c = 0
    u = np.array(float)
    u0 = np.array(float)
    r = np.array(float)
    p = np.array(float)

    ubar = np.array(float)

  #def sort_uVal(self, u):


  def read_data(self, filename):
    file = open(filename, "r")
    filetext = file.read()
    filelist = re.split(';', filetext)
    for txt in filelist:
      tmplist = re.split('=',txt)
      #print(tmplist)
      if "Multiplier" in tmplist[0]:
        self.multiplier = float(tmplist[1])
        # print(self.multiplier)
      if "NbProducts"  in tmplist[0]:
        self.nr_products = int(tmplist[1])
        # print(self.nr_products)
      if "NbRetailers" in tmplist[0]:
        self.nr_segments = int(tmplist[1])
        # print(self.nr_segments)
      if "Capacity" in tmplist[0]:
        self.c = int(tmplist[1])
        # print(self.c)
      if "Fraction" in tmplist[0]:
        self.p = eval(tmplist[1])
        # print(self.p)
      if "Preference" in tmplist[0]:
        self.u = eval(tmplist[1])
        # print(self.u)
      if "Revenue" in tmplist[0]:
        self.r = eval(tmplist[1])
        # print(self.r)
    file.close()

    self.u0 = np.ones(self.nr_segments)
    for i in range(self.nr_segments):
      for j in range(self.nr_products):
        self.u[i][j] = self.multiplier * self.u[i][j]

    # self.ubar = sorted(self.u)

    # Inside your read_data method, after assigning to self.u
    self.ubar = [(np.sort(np.array(self.u).flatten())[::-1]).tolist()]

In [ ]:
class CONICsolver:
  def __init__(self, aop: AOP):
    #Nathan's API Access Key
    params = {
      "WLSACCESSID": 'b678733f-8704-4879-b97d-d74abc85d650',
      "WLSSECRET": '432e4445-821c-4877-89ee-67f7d68bf4d4',
      "LICENSEID": 2433393,
    }
    env = gp.Env(params=params)

    self.mod = gp.Model('CONIC', env=env)
    self.aop = aop

    # Define constants
    self.r_max = dict()
    for i in range(self.aop.nr_segments):
      self.r_max[i] = max(self.aop.r[i][j] for j in range(self.aop.nr_products))

    # Define vars
    self.x = self.mod.addVars(self.aop.nr_products, vtype=gp.GRB.BINARY)
    self.y = self.mod.addVars(self.aop.nr_segments)
    self.z = self.mod.addVars(self.aop.nr_segments, self.aop.nr_products)
    self.w = self.mod.addVars(self.aop.nr_segments)

    # Define objective
    obj_expr1 = 0
    obj_expr2 = 0
    for i in range(self.aop.nr_segments):
        obj_expr1 += (
          self.aop.p[i] *
          self.aop.u0[i] *
          self.r_max[i]
        ) * self.y[i]
    for i in range(self.aop.nr_segments):
      for j in range(self.aop.nr_products):
        obj_expr2 += (
          self.aop.p[i] *
          self.aop.u[i][j] *
          (self.r_max[i] - self.aop.r[i][j])
        ) * self.z[i, j]
    self.mod.setObjective(obj_expr1 + obj_expr2, sense=gp.GRB.MINIMIZE)

    # Define constraints
    # Capacity
    self.mod.addConstr(sum(self.x[j] for j in range(self.aop.nr_products))
                       <=
                       self.aop.c
    )

    # Normalization
    self.mod.addConstrs(self.aop.u0[i] * self.y[i]
                        + sum(self.aop.u[i][j] * self.z[i, j]
                              for j in range(self.aop.nr_products)
                          )
                        ==
                        1
                        for i in range(self.aop.nr_segments)
    )

    # w definition
    self.mod.addConstrs(self.w[i]
                        ==
                        self.aop.u0[i]
                        +
                        sum(self.aop.u[i][j] * self.x[j]
                            for j in range(self.aop.nr_products)
                        )
                        for i in range(self.aop.nr_segments)
    )

    # Conic 1
    self.mod.addConstrs(self.z[i, j] * self.w[i]
                        >=
                        self.x[j] * self.x[j]
                        for i in range(self.aop.nr_segments)
                        for j in range(self.aop.nr_products)
    )

    # Conic 2
    self.mod.addConstrs(self.y[i]* self.w[i]
                        >=
                        1
                        for i in range(self.aop.nr_segments)
    )

    # McCormick Estimators
    self.mod.addConstrs(self.z[i,j]
                        >=
                        self.y[i] - (1 - self.x[j])/self.aop.u0[i]
                        for i in range(self.aop.nr_segments)
                        for j in range(self.aop.nr_products)
                        ) # zij >= yi - y^u_i * (1-xj)
    self.mod.addConstrs(self.z[i,j]
                       <=
                       self.x[j]/(self.aop.u0[i] + self.aop.u[i][j])
                       for i in range(self.aop.nr_segments)
                       for j in range(self.aop.nr_products)
                       ) # zij <= y^u_i|xj=1 * xj
    self.mod.addConstrs(self.z[i,j]
                       >=
                       self.x[j]/(self.aop.u0[i] + self.aop.u[i][j] + sum(self.aop.ubar[i][k]
                                                                      for k in range(0,(self.aop.c)-1)))
                       for i in range(self.aop.nr_segments)
                       for j in range(self.aop.nr_products)
                       ) # zij >= y^l_i|xj=1 * xj
    self.mod.addConstrs(self.z[i,j]
                       <=
                       self.y[i] - ((1 - self.x[j])/(self.aop.u0[i] + sum(self.aop.ubar[i][k]
                                                                           for k in range(self.aop.c))))
                       for i in range(self.aop.nr_segments)
                       for j in range(self.aop.nr_products)
                       ) # zij<= yi - y^u_i *(1-xj)

  def solve(self):
    self.mod.optimize()

In [ ]:
aop = AOP()
aop.read_data("200_20_set1.dat")
#aop.read_data("100_100_hard_set5.dat")

print("data read")
#milpsolver = MILPsolver(aop)
#milpsolver.solve()

# Debugging
print(range(aop.c))
print(range(0,(aop.c)-1))
print(aop.u)
print(aop.ubar)

# conicsolver = CONICsolver(aop)
# conicsolver.solve()

data read
range(0, 10)
range(0, 9)
[[0.05176, 0.018600000000000002, 0.17236, 0.15314000000000003, 0.04224, 0.07046000000000001, 0.17744000000000001, 0.1107, 0.10176000000000002, 0.13346, 0.07132, 0.19052000000000002, 0.11532, 0.15822000000000003, 0.06228, 0.10874, 0.14830000000000002, 0.08534000000000001, 0.0567, 0.01508, 0.00234, 0.01566, 0.06808, 0.10528, 0.19198, 0.14754, 0.17674, 0.00464, 0.09796, 0.06866, 0.043000000000000003, 0.16484000000000001, 0.0097, 0.11160000000000002, 0.10966000000000001, 0.04184, 0.18580000000000002, 0.03066, 0.09248, 0.15146, 0.05828, 0.16762, 0.12322, 0.04172000000000001, 0.09526000000000001, 0.16298, 0.15506, 0.17852, 0.17226, 0.04812, 0.14978, 0.08142, 0.13078, 0.16838, 0.119, 0.15592, 0.02374, 0.00166, 0.04642, 0.05002, 0.17302, 0.13576, 0.05558, 0.00204, 0.00026, 0.05802000000000001, 0.1545, 0.07056, 0.05978, 0.014940000000000002, 0.17416, 0.022080000000000002, 0.10548, 0.09330000000000001, 0.15146, 0.16636, 0.17858000000000002, 0.06602000000000001,